In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import json
from helpers import *
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 8})
import itertools
codeToggler()

In [ ]:
(df, survey, objNames, trial_types) = loadData()
df = pd.read_json('data/trials-with-scores.json')

# Is there an effect of AQ score?

In [ ]:
aqs = []
for subject_id in df['subject_id']:
    aq = survey[survey['subject_id'] == subject_id]
    if(len(aq) != 0):
        aqs += [aq.iloc[0,:]['AQ_score']]
    else:
        print(aq)
        aqs += [0]
        
df['AQ_score'] = aqs

In [ ]:
subscales = []
for i, response in survey.iterrows():
    selections = json.loads(response['AQ_selections'])
    __df = pd.DataFrame([(int(q), s) for q, s in selections.iteritems()], columns=['question', 'selection'])
    __df = __df.sort_values(by='question').set_index('question')
    __df['points'] = AQPoints(__df['selection'])
    subscales += [AQSubscaleScores(__df['points'])]

In [ ]:
__df = pd.DataFrame(subscales)
survey = survey.reset_index()
for col in __df.columns:
    survey[col] = __df[col]
survey = survey.set_index('survey_id')

In [ ]:
# survey.to_json('data/survey-with-subscales.json')

In [ ]:
# Add subscales to df
for subscale in ['attention_switching','communication','detail','imagination','social']:
    values = []
    for subject_id in df['subject_id']:
        part = survey[survey['subject_id'] == subject_id]
        if(len(part) != 0):
            values += [part.iloc[0,:][subscale]]
        else:
            print(aq)
            values += [0]

    df[subscale] = values

In [ ]:
for subscale in ['attention_switching','communication','detail','imagination','social']:    
    __df = df.groupby('subject_id').mean()[['analogy_score', subscale]].sort_values(by=subscale)
    __df[__df['analogy_score'] < 3].plot(kind='scatter', x=subscale, y='analogy_score', 
                                        figsize=(4.5,2.5), yticks=[0,.5,1,1.5,2])

    plt.ylabel('Experiment score')
    plt.xlabel('AQ subscale %s' % subscale.replace('_', ' '));

    # Title
    plt.title('AQ subscale %s and experiment score' % subscale, y=1.08);

    # Save plot
    plt.tight_layout()
    plt.show()
    
    print('Spearman correlation:')
    print(stats.spearmanr(__df['analogy_score'], __df[subscale]))